# Create grid
This script contains a method for creating a grid out of a layer file

**Import libraries**

In [20]:
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely.geometry
import time

print('Libraries imported successfully')

ModuleNotFoundError: No module named 'geopandas'

In [21]:
seg_start=time.time() #For checking ejecution time

**Define directories**

 - <b>path_to_layer</b>: path to shp file of the desire extent
 
 - <b>output</b>: output folder where the grid will be saved
 
 - <b>spacing</b>: spacing between polygons in the grid
  
 - <b>epsg</b>: define epsg 
 
 - <b>buffer</b>: intersection of polygons

In [ ]:
tmp = '/home/jovyan/work/'

path_to_layer = f'{tmp}Dataset_files/comunidad_valenciana.shp'  
output = f'{tmp}Dataset_files/abc_comunidad_valenciana.shp'
spacing = 16000
epsg = 25830 
buffer = True 

Define **size of the extent** to be sure the bounding box created has all the set point inside and it is multiple of the spacing.

In [ ]:
gdf = gpd.read_file(path_to_layer)
xmin, ymin, xmax, ymax = gdf.total_bounds # bounds of the total geometry

ytop = np.ceil(np.ceil(ymax) / spacing) * spacing
ybottom = np.floor(np.floor(ymin) / spacing) * spacing
xright = np.ceil(np.ceil(xmax) / spacing) * spacing
xleft = np.floor(np.floor(xmin) / spacing) * spacing

# Defining number of rows and columns
rows = int((ytop - ybottom) / spacing)
cols = int((xright - xleft) / spacing)

A for loop for **creating all polygons** of the grid

In [ ]:
polygons = []
it = 0
listfid = []

for i in np.arange(xleft, xright, spacing):
    xleft = i
    xright = xleft + spacing
    ytop_backup = ytop
    for j in np.arange(ytop, ybottom, -spacing):
        ytop = j
        ybottom = ytop - spacing

        polygon = shapely.geometry.Polygon([
            (xleft, ytop),
            (xright, ytop),
            (xright, ybottom),
            (xleft, ybottom)
        ]
        )
        polygons.append(polygon)
        listfid.append(it)
        it += 1
    ytop = ytop_backup

Set metadata and write it into the disk

In [ ]:
srs = f"epsg:{epsg}"
fid = pd.DataFrame({"fid_id": listfid})
grid = gpd.GeoDataFrame(fid, geometry=polygons, crs={"init": srs})

print("Writing grid into disk")
grid.to_file(output, driver="GPKG")

In [ ]:
if buffer:
    buf = grid.geometry.buffer(buffer)
    envelope = buf.envelope
    return(envelope)
else:
    return(grid)

In [ ]:
print('Grid finished successfully in {time.time() - seg_start} s')